In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import pandas as pd
import scipy.optimize as opt
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import re
import matplotlib.backends.backend_pdf
import os
import time
from rdkit import Chem
from mordred import Calculator, descriptors
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from matplotlib import gridspec
from PIL import Image
from random import Random
import scipy.stats as stt

2022-09-18 15:30:34.268 INFO    rdkit: Enabling RDKit 2020.09.1 jupyter extensions


In [2]:

def split_data(dataset):
    dataset=dataset.sample(frac=1)
    length= len(dataset)
    eightypercent= int(length*0.8)
    trainingset= dataset[:eightypercent]
    testset=dataset[eightypercent:]
    return trainingset, testset

def re_round(li, _prec=3):
    ''' Rounds argument li to desired number of decimals (_prec) '''
    try:
         return round(li, _prec)
    except TypeError:
         return type(li)(re_round(x, _prec) for x in li)
             
def findstr(test_str,test_sub):
    res = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    return res
    
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def split_data(dataset):
    dataset=dataset.sample(frac=1)
    length= len(dataset)
    eightypercent= int(length*0.8)
    trainingset= dataset[:eightypercent]
    testset=dataset[eightypercent:]
    return trainingset, testset

def fit_tm_model(predictors,*parameter):
    parameters= parameter
    return eval(model_form)

def fit_tm_model_err(predictors,*parameter):
    parameters= parameter[0]
    return eval(model_form)

def make_plots(dataset_test,dataset_train,letters_in_use, dataset_name):
    Tbuffer= 25
    lowerT =min ( min(dataset_test['T_m (K)']),min(dataset_train['T_m (K)']),min(fit_tm_model_err(dataset_test,letters_in_use)),min (fit_tm_model_err(dataset_train,letters_in_use))) - 273
    lowerT=lowerT- Tbuffer

    higherT =max( max(dataset_test['T_m (K)']),max(dataset_train['T_m (K)'])
    ,max(fit_tm_model_err(dataset_test,
    letters_in_use)),max(fit_tm_model_err(dataset_train,letters_in_use))) - 273
    higherT=higherT+Tbuffer

    
    fig, ax=plt.subplots(figsize=(4,4), dpi=300)
    
    #ax.figure(figsize=(4,4), dpi=300)
    ax.plot(dataset_train['T_m (K)']-273,fit_tm_model_err(dataset_train,
    letters_in_use)-273,'ko',markersize=4)
    ax.plot(dataset_test['T_m (K)']-273,fit_tm_model_err(dataset_test,
    letters_in_use)-273,'ro',markersize=4)
    ax.plot([lowerT,higherT],[lowerT,higherT],color=((0.6,0.6,0.6)))
    ax.set_ylabel('Predicted $T_m$ (°C)')
    ax.set_xlabel('Experimental $T_m$ (°C)')
    #ax.set_title('Calculated vs Experimental $T_m$ for '+ model_form_name + ' - ' + dataset_name +'\n Test Error: ' + str(avg_model_err[0])+ '\n Train Error: ' + str(avg_model_err[1]) + '\n RMSE Test:'+ str(rmse_err[0])+ '\n RMSE Train:'+str(rmse_err[1]))
    ax.legend(('Training Set','Test Set'))
    ax.set_xlim([lowerT,higherT])
    ax.set_ylim([lowerT,higherT])
    return fig

# function to calculate root mean square error
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# Function for rounding tuples in confidence intervals - copied from online
def re_round(li, _prec=3):
    ''' Rounds argument li to desired number of decimals (_prec) '''
    try:
         return round(li, _prec)
    except TypeError:
         return type(li)(re_round(x, _prec) for x in li)

def vbt_model_automated(dataset_dictionary, dataset_name, model_form, starting_guesses, number_of_runs):
    '''Generates a dictionary of VBT model plot, Avg Absolute Error, RMSE Error, and Parameter values'''
    all_possible_predictors= ['tau', 'V_m (nm3)', 'sigma','Eccentricity(Eal)', 'Eccentricity(Ear)']
    letters_in_use= ['a','b','c','d','f','g','h','k','l','m']
    predicted= ['T_m (K)']
    used_predictors= []
    # This for loop finds which parameters are used in the model form and adds them to a list of used predictors
    for i in range(len(all_possible_predictors)):
        this_predictor= all_possible_predictors[i]
        if findstr(model_form, this_predictor):
            used_predictors.append(this_predictor)
    used_predictors= used_predictors+predicted
    num_parameters= len(findstr(model_form, 'parameter'))

    if num_parameters!=len(starting_guesses):
        raise Exception("Number of starting guesses doesn't match the number of parameters")

    letters_in_use=letters_in_use[0:num_parameters]
    num_parameters= list(range(num_parameters))
    train_avg_err=[]
    test_avg_err=[]
    train_rmse_err=[]
    test_rmse_err=[]
    count=0
    parameters_from_runs=np.zeros((number_of_runs,len(num_parameters)))
    dataset= dataset_dictionary[dataset_name]
    #dataset_length= len(dataset)
    predictors=dataset[used_predictors].astype('float64')
    while count < number_of_runs:
        [dataset_train,dataset_test]=split_data(predictors)
        (letters_in_use,_)=opt.curve_fit(fit_tm_model,dataset_train,dataset_train['T_m (K)'],(starting_guesses))
        parameters_from_runs[count,:]=letters_in_use
        train_avg_err.append(statistics.mean(np.absolute(fit_tm_model_err(dataset_train,letters_in_use)-dataset_train['T_m (K)'])))
        test_avg_err.append(statistics.mean(np.absolute(fit_tm_model_err(dataset_test,letters_in_use)-dataset_test['T_m (K)'])))
        train_rmse_err.append(rmse((fit_tm_model_err(dataset_train,letters_in_use)),dataset_train['T_m (K)']))
        test_rmse_err.append(rmse((fit_tm_model_err(dataset_test,letters_in_use)),dataset_test['T_m (K)']))
        count = count+1

    fig=make_plots(dataset_test,dataset_train,letters_in_use,dataset_name)
    
    ci_int_val=0.95
    dec_points=3

    ind_names=[]
    for i in range(1,num_runs+1):
        ind_names.append('Run ' + str(i))#print(rmse_err)
    errors = pd.DataFrame(list(zip(train_avg_err,test_avg_err,train_rmse_err,test_rmse_err)),columns=['Train AAE','Test AAE','Train RMSE','Test RMSE'],index=ind_names).round(decimals=dec_points)

    mean_list=[]
    ci_list=[]

    for column in errors:
        mean_list.append(np.mean(errors[column]))
        ci_list.append(stt.t.interval(ci_int_val,len(errors[column])-1, np.mean(errors[column]), stt.sem(errors[column]))) 
    errors.loc['Mean'] = re_round(mean_list,dec_points)
    errors.loc['95% CI'] = re_round(ci_list,dec_points)

    calc_parameters = pd.DataFrame(np.around(parameters_from_runs,dec_points),index=ind_names)
    mean_list_p = []
    ci_list_p = []
    for column in calc_parameters:
        mean_list_p.append(np.mean(calc_parameters[column]))
        ci_list_p.append(stt.t.interval(ci_int_val,len(calc_parameters[column])-1, np.mean(calc_parameters[column]), stt.sem(calc_parameters[column])))
    calc_parameters.loc['Mean'] = re_round(mean_list_p,dec_points)
    calc_parameters.loc['95% CI'] = re_round(ci_list_p,dec_points)
    model_dict = {
        'Plot': fig,
        'Errors': errors,
        'Parameters': calc_parameters
    }
    plt.close()
    return model_dict


In [196]:



def ml_model_vbt_features_multipleruns(vbt_dataset_dict,dataset_key,alpha=1,number_of_runs=5):
    
    full_dataset = vbt_dataset_dict[dataset_key][['sigma','tau','V_m (nm3)','T_m (K)','Eccentricity(Ear)','Eccentricity(Eal)']]

    train_avg_err=[]
    test_avg_err=[]
    train_rmse_err=[]
    test_rmse_err=[]
    runs = ["Run " + str(i+1) for i in range(number_of_runs)]
    weights = pd.DataFrame(columns = ['sigma','tau','V_m (nm3)','Eccentricity(Ear)','Eccentricity(Eal)'],index=runs)

    count = 1
    while count < number_of_runs + 1:
        [trainset,testset] = split_data(full_dataset)
        y = (trainset['T_m (K)']-273).tolist()
        y_test = (testset['T_m (K)']-273).tolist()
        trainset = trainset.drop(columns = ['T_m (K)'])
        testset = testset.drop(columns = ['T_m (K)'])
        X = preprocessing.scale(trainset)
        scaler = preprocessing.StandardScaler().fit(trainset)
        X_test = scaler.transform(testset)
        rr = Ridge(alpha=alpha)
        rr.fit(X, y)
        w = rr.coef_
        intercept = rr.intercept_
        
        train_avg_err.append(np.mean(np.abs(y-(np.dot(X,w)+intercept))))
        test_avg_err.append(np.mean(np.abs(y_test-(np.dot(X_test,w)+intercept))))
        train_rmse_err.append(np.sqrt(((y - (np.dot(X,w)+intercept)) ** 2).mean()))
        test_rmse_err.append(np.sqrt(((y_test-(np.dot(X_test,w)+intercept))**2).mean()))
        
        weights.loc["Run " + str(count),:] = w
        count = count + 1
    
    ci_int_val=0.95
    dec_points=3

    errors = pd.DataFrame(list(zip(train_avg_err,test_avg_err,train_rmse_err,test_rmse_err)),columns=['Train AAE','Test AAE','Train RMSE','Test RMSE'],index=runs).round(decimals=dec_points)

    mean_list = [np.mean(errors[column]) for column in errors]
    ci_list = [stt.t.interval(ci_int_val,len(errors[column])-1, np.mean(errors[column]), stt.sem(errors[column])) for column in errors]
    errors.loc['Mean'] = np.around(mean_list,dec_points)
    errors.loc['95% CI'] = re_round(ci_list,dec_points)

    mean_list = [np.mean(weights[column]) for column in weights]
    ci_list = [stt.t.interval(ci_int_val,len(weights[column])-1, np.mean(weights[column]), stt.sem(weights[column])) for column in weights]
    weights.loc['Mean'] = np.around(mean_list,dec_points)
    weights.loc['95% CI'] = re_round(ci_list,dec_points)

    # Make plot
    model_plot=plt.figure(figsize=(4,4), dpi=300)
    ax1 = plt.gca()
    ml_plot_train_points=ax1.scatter(y,np.dot(X,w)+intercept,c='k',alpha=1,linewidth=0)
    ax1.plot([-273,2000],[-273,2000],'k--',lw=1)
    ml_plot_test_points=ax1.scatter(y_test,np.dot(X_test,w)+intercept,c='r',alpha=1,linewidth=0)
    lims = [min(y+y_test)-15,max(y+y_test)+15]
    for theaxis in [ax1]:        
        theaxis.set_aspect(1)
        theaxis.set_xlim(lims)
        theaxis.set_ylim(lims)
        theaxis.set_xlabel(r"Experimental $T_m$ ($^{\circ}$C)")
        theaxis.set_ylabel(r"Predicted $T_m$ ($^{\circ}$C)")
        
        for item in ([theaxis.xaxis.label, theaxis.yaxis.label, theaxis.yaxis.get_offset_text(), theaxis.xaxis.get_offset_text()]):
            item.set_fontsize(12)
        for item in (theaxis.get_xticklabels() + theaxis.get_yticklabels()):
            item.set_fontsize(10)
    plt.legend((ml_plot_train_points,ml_plot_test_points),('Training Set','Test Set'))    
    plt.gcf().subplots_adjust(left=0.2,top=0.95,bottom=0.15,right=0.95)
   
    plt.close()
    
    ml_model_dict={
        'Plot': model_plot,
        'Errors': errors,
        'Model Coefficients': weights
    }
    return ml_model_dict

def rf_model_vbt_features_multipleruns(vbt_dataset_dict,dataset_key,number_of_runs=5):
    
    full_dataset = vbt_dataset_dict[dataset_key][['sigma','tau','V_m (nm3)','T_m (K)','Eccentricity(Ear)','Eccentricity(Eal)']]

    train_avg_err=[]
    test_avg_err=[]
    train_rmse_err=[]
    test_rmse_err=[]
    runs = ["Run " + str(i+1) for i in range(number_of_runs)]
    weights = pd.DataFrame(columns = ['sigma','tau','V_m (nm3)','Eccentricity(Ear)','Eccentricity(Eal)'],index=runs)

    count = 1
    while count < number_of_runs + 1:
        [trainset,testset] = split_data(full_dataset)
        y = (trainset['T_m (K)']-273).tolist()
        y_test = (testset['T_m (K)']-273).tolist()
        trainset = trainset.drop(columns = ['T_m (K)'])
        testset = testset.drop(columns = ['T_m (K)'])
        X = preprocessing.scale(trainset)
        scaler = preprocessing.StandardScaler().fit(trainset)
        X_test = scaler.transform(testset)
        rf = RandomForestRegressor()
        rf.fit(X, y)
        # w = rr.coef_
        # intercept = rr.intercept_
        
        train_avg_err.append(np.mean(np.abs(y-(rf.predict(X)))))
        test_avg_err.append(np.mean(np.abs(y_test-(rf.predict(X_test)))))
        train_rmse_err.append(np.sqrt(((y - (rf.predict(X))) ** 2).mean()))
        test_rmse_err.append(np.sqrt(((y_test-(rf.predict(X_test)))**2).mean()))
        
        weights.loc["Run " + str(count),:] = rf.feature_importances_
        count = count + 1
    
    ci_int_val=0.95
    dec_points=3

    errors = pd.DataFrame(list(zip(train_avg_err,test_avg_err,train_rmse_err,test_rmse_err)),columns=['Train AAE','Test AAE','Train RMSE','Test RMSE'],index=runs).round(decimals=dec_points)

    mean_list = [np.mean(errors[column]) for column in errors]
    ci_list = [stt.t.interval(ci_int_val,len(errors[column])-1, np.mean(errors[column]), stt.sem(errors[column])) for column in errors]
    errors.loc['Mean'] = np.around(mean_list,dec_points)
    errors.loc['95% CI'] = re_round(ci_list,dec_points)

    mean_list = [np.mean(weights[column]) for column in weights]
    ci_list = [stt.t.interval(ci_int_val,len(weights[column])-1, np.mean(weights[column]), stt.sem(weights[column])) for column in weights]
    weights.loc['Mean'] = np.around(mean_list,dec_points)
    weights.loc['95% CI'] = re_round(ci_list,dec_points)

    # Make plot
    model_plot=plt.figure(figsize=(4,4), dpi=300)
    ax1 = plt.gca()
    ml_plot_train_points=ax1.scatter(y,rf.predict(X),c='k',alpha=1,linewidth=0)
    ax1.plot([-273,2000],[-273,2000],'k--',lw=1)
    ml_plot_test_points=ax1.scatter(y_test,rf.predict(X_test),c='r',alpha=1,linewidth=0)
    lims = [min(y+y_test)-15,max(y+y_test)+15]
    for theaxis in [ax1]:        
        theaxis.set_aspect(1)
        theaxis.set_xlim(lims)
        theaxis.set_ylim(lims)
        theaxis.set_xlabel(r"Experimental $T_m$ ($^{\circ}$C)")
        theaxis.set_ylabel(r"Predicted $T_m$ ($^{\circ}$C)")
        
        for item in ([theaxis.xaxis.label, theaxis.yaxis.label, theaxis.yaxis.get_offset_text(), theaxis.xaxis.get_offset_text()]):
            item.set_fontsize(12)
        for item in (theaxis.get_xticklabels() + theaxis.get_yticklabels()):
            item.set_fontsize(10)
    plt.legend((ml_plot_train_points,ml_plot_test_points),('Training Set','Test Set'))    
    plt.gcf().subplots_adjust(left=0.2,top=0.95,bottom=0.15,right=0.95)
   
    plt.close()
    
    ml_model_dict={
        'Plot': model_plot,
        'Errors': errors,
        'Feature Importances': weights
    }
    return ml_model_dict

def ml_model_multipleruns(ml_dataset_dict, dataset_key, alpha=100, number_of_runs=5, do_featurization = False):  
    if do_featurization:

        working_ML_dataset=ml_dataset_dict[dataset_key]
        mols_full=[Chem.MolFromSmiles(m) for m in working_ML_dataset.SMILES.tolist() if Chem.MolFromSmiles(m) != None]
        calc = Calculator(descriptors, ignore_3D=True)
        mordredresults = calc.pandas(mols_full)
        working_ML_dataset = working_ML_dataset.join(mordredresults,how='inner')
        # save to the appropriate file name depending on which dataset you're looking at
        if dataset_key=='Quinones':
            working_ML_dataset.to_csv('Data Files/featurized_bq.csv',index=False)
        elif dataset_key=='Hydroquinones':
            working_ML_dataset.to_csv('Data Files/featurized_hq.csv',index=False)
        else:
            print('Not a valid dataset key')            
    else:
        if dataset_key=='Quinones':
            working_ML_dataset=pd.read_csv('Data Files/featurized_bq.csv',low_memory=False)
        elif dataset_key=='Hydroquinones':
            working_ML_dataset=pd.read_csv('Data Files/featurized_hq.csv',low_memory=False)

    train_avg_err=[]
    test_avg_err=[]
    train_rmse_err=[]
    test_rmse_err=[]
    runs = ["Run " + str(i+1) for i in range(number_of_runs)]
    weights = pd.DataFrame(columns = working_ML_dataset.columns.tolist(),index=runs)

    count = 1
    while count < number_of_runs + 1:
        [trainset,testset] = split_data(working_ML_dataset)
        trainset = trainset.reset_index()
        testset = testset.reset_index()        
        
        ######
        ## Standardization (want each feature to be a gaussian with zero mean and unit variance)
        ######

        # drop non-numeric columns and ones for the melting point, so we only have columns of features
        # I don't know why, but LogP caused a problem during the standardization - dropping for now, but have to figure out
        # Now dropping everything but MW from the Reaxys, since we don't have it for the Na and K salts
        # columns_to_drop_from_reaxys = ['InChI Key','SMILES','Type of Substance','mp_mean','mp_std','LogP','H Bond Donors','H Bond Acceptors','Rotatable Bonds','TPSA','Lipinski Number','Veber Number','MAXdO','MINdO']
        columns_to_drop = ['InChI Key','SMILES','Type of Substance','mp_mean','mp_std','LogP','H Bond Donors','H Bond Acceptors','Rotatable Bonds','TPSA','Lipinski Number','Veber Number','MAXdO','MINdO','index']

        # if dataset_key=='Quinones':
        #     columns_to_drop_thatgavetrouble = ['MAXdO','MINdO']
        #     # for bq these gave trouble: ['Unnamed: 0','MAXdO','MINdO']
        # elif dataset_key=='Hydroquinones':
        #     columns_to_drop_thatgavetrouble = ['MAXdO','MINdO']

        # columns_to_drop = columns_to_drop_from_reaxys + columns_to_drop_thatgavetrouble
        trainset_s = trainset.drop(columns=columns_to_drop)
        testset_s = testset.drop(columns=columns_to_drop)

        # drop columns where there is an error from mordred
        #print('Started with '+str(len(trainset_s.columns))+' features')
        trainset_s = trainset_s.select_dtypes(include=['float64','int'])

        #print('After dropping columns with mordred errors, have '+str(len(trainset_s.columns))+' features')

        # drop the same columns from the test set
        testset_s = testset_s[trainset_s.columns]

        # finally, do the standardization
        X = preprocessing.scale(trainset_s)
        # apply the same standardization to the test set
        scaler = preprocessing.StandardScaler().fit(trainset_s)
        X_test = scaler.transform(testset_s)

        ######
        ## Ridge regression
        ######

        y = trainset.mp_mean.tolist()
        y_test = testset.mp_mean.tolist()

        rr = Ridge(alpha=alpha)
        rr.fit(X, y)
        w = rr.coef_
        intercept = rr.intercept_

        # avg_abs_err = np.zeros(2)
        # rmse_err = np.zeros(2)
        # The test set is at the 0 index and the training set is at the 1 index to match the convention in the other model 
        train_avg_err.append(np.mean(np.abs(y-(np.dot(X,w)+intercept))))
        test_avg_err.append(np.mean(np.abs(y_test-(np.dot(X_test,w)+intercept))))
        train_rmse_err.append(np.sqrt(((y - (np.dot(X,w)+intercept)) ** 2).mean()))
        test_rmse_err.append(np.sqrt(((y_test-(np.dot(X_test,w)+intercept))**2).mean()))

        if count == 1:
            weights = pd.DataFrame(index=runs,columns=trainset_s.columns)
        # coeffs = pd.DataFrame(data={'label':trainset_s.columns, 'w':w, 'w_abs':np.abs(w)})
        weights.loc["Run " + str(count),:] = w

        count = count + 1
    
    ci_int_val=0.95
    dec_points=3
    
    errors = pd.DataFrame(list(zip(train_avg_err,test_avg_err,train_rmse_err,test_rmse_err)),columns=['Train AAE','Test AAE','Train RMSE','Test RMSE'],index=runs).round(decimals=dec_points)

    mean_list = [np.mean(errors[column]) for column in errors]
    ci_list = [stt.t.interval(ci_int_val,len(errors[column])-1, np.mean(errors[column]), stt.sem(errors[column])) for column in errors]
    errors.loc['Mean'] = np.around(mean_list,dec_points)
    errors.loc['95% CI'] = re_round(ci_list,dec_points)

    mean_list = [np.mean(weights[column]) for column in weights]
    ci_list = [stt.t.interval(ci_int_val,len(weights[column])-1, np.mean(weights[column]), stt.sem(weights[column])) for column in weights]
    weights.loc['Mean'] = np.around(mean_list,dec_points)
    weights.loc["Absolute Mean"] = np.abs(weights.loc["Mean"])
    weights.loc['95% CI'] = re_round(ci_list,dec_points)
    # Sort by the mean weight in descending order
    weights.sort_values(by=["Absolute Mean"],axis=1,ascending=False,inplace=True)

    # Make plot
    model_plot=plt.figure(figsize=(4,4), dpi=300)
    ax1 = plt.gca()
    ml_plot_train_points=ax1.scatter(y,np.dot(X,w)+intercept,s=5,c='k',alpha=0.7,linewidth=0)
    ax1.plot([-273,2000],[-273,2000],'k--',lw=1)
    ml_plot_test_points=ax1.scatter(y_test,np.dot(X_test,w)+intercept,s=5,c='r',alpha=0.7,linewidth=0)

    lims = [min(y+y_test)-5,max(y+y_test)+5]

    for theaxis in [ax1]:
        
        theaxis.set_aspect(1)
        theaxis.set_xlim(lims)
        theaxis.set_ylim(lims)

        theaxis.set_xlabel(r"Experimental $T_m$ ($^{\circ}$C)")
        theaxis.set_ylabel(r"Predicted $T_m$ ($^{\circ}$C)")
        
        for item in ([theaxis.xaxis.label, theaxis.yaxis.label, theaxis.yaxis.get_offset_text(), theaxis.xaxis.get_offset_text()]):
            item.set_fontsize(12)
        for item in (theaxis.get_xticklabels() + theaxis.get_yticklabels()):
            item.set_fontsize(10)
    plt.legend((ml_plot_train_points,ml_plot_test_points),('Training Set','Test Set'))    
    plt.gcf().subplots_adjust(left=0.2,top=0.95,bottom=0.15,right=0.95)
   
    plt.close()

    ml_model_dict={
        'Plot': model_plot,
        'Errors': errors,
        'Model Coefficients': weights
    }
    return ml_model_dict

def rf_model_multipleruns(ml_dataset_dict, dataset_key, number_of_runs=5, do_featurization = False):  
    if do_featurization:

        working_ML_dataset=ml_dataset_dict[dataset_key]
        mols_full=[Chem.MolFromSmiles(m) for m in working_ML_dataset.SMILES.tolist() if Chem.MolFromSmiles(m) != None]
        calc = Calculator(descriptors, ignore_3D=True)
        mordredresults = calc.pandas(mols_full)
        working_ML_dataset = working_ML_dataset.join(mordredresults,how='inner')
        # save to the appropriate file name depending on which dataset you're looking at
        if dataset_key=='Quinones':
            working_ML_dataset.to_csv('Data Files/featurized_bq.csv',index=False)
        elif dataset_key=='Hydroquinones':
            working_ML_dataset.to_csv('Data Files/featurized_hq.csv',index=False)
        else:
            print('Not a valid dataset key')            
    else:
        if dataset_key=='Quinones':
            working_ML_dataset=pd.read_csv('Data Files/featurized_bq.csv',low_memory=False)
        elif dataset_key=='Hydroquinones':
            working_ML_dataset=pd.read_csv('Data Files/featurized_hq.csv',low_memory=False)

    train_avg_err=[]
    test_avg_err=[]
    train_rmse_err=[]
    test_rmse_err=[]
    runs = ["Run " + str(i+1) for i in range(number_of_runs)]
    weights = pd.DataFrame(columns = working_ML_dataset.columns.tolist(),index=runs)

    count = 1
    while count < number_of_runs + 1:
        [trainset,testset] = split_data(working_ML_dataset)
        trainset = trainset.reset_index()
        testset = testset.reset_index()        
        
        ######
        ## Standardization (want each feature to be a gaussian with zero mean and unit variance)
        ######

        columns_to_drop = ['InChI Key','SMILES','Type of Substance','mp_mean','mp_std','LogP','H Bond Donors','H Bond Acceptors','Rotatable Bonds','TPSA','Lipinski Number','Veber Number','MAXdO','MINdO','index']

        trainset_s = trainset.drop(columns=columns_to_drop)
        testset_s = testset.drop(columns=columns_to_drop)

        # drop columns where there is an error from mordred
        trainset_s = trainset_s.select_dtypes(include=['float64','int'])

        # drop the same columns from the test set
        testset_s = testset_s[trainset_s.columns]

        # finally, do the standardization
        X = preprocessing.scale(trainset_s)
        # apply the same standardization to the test set
        scaler = preprocessing.StandardScaler().fit(trainset_s)
        X_test = scaler.transform(testset_s)

        ######
        ## Random Forest regression
        ######

        y = trainset.mp_mean.tolist()
        y_test = testset.mp_mean.tolist()

        rf = RandomForestRegressor(n_jobs=-1) # In theory, this speeds up the process
        rf.fit(X, y)
        # w = rr.coef_
        # intercept = rr.intercept_

        train_avg_err.append(np.mean(np.abs(y-rf.predict(X))))
        test_avg_err.append(np.mean(np.abs(y_test-rf.predict(X_test))))
        train_rmse_err.append(np.sqrt(((y - rf.predict(X)) ** 2).mean()))
        test_rmse_err.append(np.sqrt(((y_test- rf.predict(X_test))**2).mean()))

        if count == 1:
            weights = pd.DataFrame(index=runs,columns=trainset_s.columns)
        weights.loc["Run " + str(count),:] = rf.feature_importances_

        count = count + 1
    
    ci_int_val=0.95
    dec_points=3
    
    errors = pd.DataFrame(list(zip(train_avg_err,test_avg_err,train_rmse_err,test_rmse_err)),columns=['Train AAE','Test AAE','Train RMSE','Test RMSE'],index=runs).round(decimals=dec_points)

    mean_list = [np.mean(errors[column]) for column in errors]
    ci_list = [stt.t.interval(ci_int_val,len(errors[column])-1, np.mean(errors[column]), stt.sem(errors[column])) for column in errors]
    errors.loc['Mean'] = np.around(mean_list,dec_points)
    errors.loc['95% CI'] = re_round(ci_list,dec_points)

    mean_list = [np.mean(weights[column]) for column in weights]
    ci_list = [stt.t.interval(ci_int_val,len(weights[column])-1, np.mean(weights[column]), stt.sem(weights[column])) for column in weights]
    weights.loc['Mean'] = np.around(mean_list,dec_points)
    weights.loc["Absolute Mean"] = np.abs(weights.loc["Mean"])
    weights.loc['95% CI'] = re_round(ci_list,dec_points)
    # Sort by the mean weight in descending order
    weights.sort_values(by=["Absolute Mean"],axis=1,ascending=False,inplace=True)

    # Make plot
    model_plot=plt.figure(figsize=(4,4), dpi=300)
    ax1 = plt.gca()
    ml_plot_train_points=ax1.scatter(y,rf.predict(X),s=5,c='k',alpha=0.7,linewidth=0)
    ax1.plot([-273,2000],[-273,2000],'k--',lw=1)
    ml_plot_test_points=ax1.scatter(y_test,rf.predict(X_test),s=5,c='r',alpha=0.7,linewidth=0)

    lims = [min(y+y_test)-5,max(y+y_test)+5]

    for theaxis in [ax1]:
        
        theaxis.set_aspect(1)
        theaxis.set_xlim(lims)
        theaxis.set_ylim(lims)

        theaxis.set_xlabel(r"Experimental $T_m$ ($^{\circ}$C)")
        theaxis.set_ylabel(r"Predicted $T_m$ ($^{\circ}$C)")
        
        for item in ([theaxis.xaxis.label, theaxis.yaxis.label, theaxis.yaxis.get_offset_text(), theaxis.xaxis.get_offset_text()]):
            item.set_fontsize(12)
        for item in (theaxis.get_xticklabels() + theaxis.get_yticklabels()):
            item.set_fontsize(10)
    plt.legend((ml_plot_train_points,ml_plot_test_points),('Training Set','Test Set'))    
    plt.gcf().subplots_adjust(left=0.2,top=0.95,bottom=0.15,right=0.95)
   
    plt.close()

    ml_model_dict={
        'Plot': model_plot,
        'Errors': errors,
        'Feature Importances': weights
    }
    return ml_model_dict

def ml_model_vbt_features_tau_inputs_multipleruns(vbt_dataset_dict,dataset_key,alpha=1,number_of_runs=5):
    
    full_dataset = vbt_dataset_dict[dataset_key][['sigma','SP3 (LIN)','SP3 (BR)','SP2','Ring','V_m (nm3)','T_m (K)','Eccentricity(Ear)','Eccentricity(Eal)']]

    train_avg_err=[]
    test_avg_err=[]
    train_rmse_err=[]
    test_rmse_err=[]
    runs = ["Run " + str(i+1) for i in range(number_of_runs)]
    weights = pd.DataFrame(columns = ['sigma','SP3 (LIN)','SP3 (BR)','SP2','Ring','V_m (nm3)','Eccentricity(Ear)','Eccentricity(Eal)'],index=runs)

    count = 1
    while count < number_of_runs + 1:
        [trainset,testset] = split_data(full_dataset)
        y = (trainset['T_m (K)']-273).tolist()
        y_test = (testset['T_m (K)']-273).tolist()
        trainset = trainset.drop(columns = ['T_m (K)'])
        testset = testset.drop(columns = ['T_m (K)'])
        X = preprocessing.scale(trainset)
        scaler = preprocessing.StandardScaler().fit(trainset)
        X_test = scaler.transform(testset)
        rr = Ridge(alpha=alpha)
        rr.fit(X, y)
        w = rr.coef_
        intercept = rr.intercept_
        
        train_avg_err.append(np.mean(np.abs(y-(np.dot(X,w)+intercept))))
        test_avg_err.append(np.mean(np.abs(y_test-(np.dot(X_test,w)+intercept))))
        train_rmse_err.append(np.sqrt(((y - (np.dot(X,w)+intercept)) ** 2).mean()))
        test_rmse_err.append(np.sqrt(((y_test-(np.dot(X_test,w)+intercept))**2).mean()))
        
        weights.loc["Run " + str(count),:] = w
        count = count + 1
    
    ci_int_val=0.95
    dec_points=3

    errors = pd.DataFrame(list(zip(train_avg_err,test_avg_err,train_rmse_err,test_rmse_err)),columns=['Train AAE','Test AAE','Train RMSE','Test RMSE'],index=runs).round(decimals=dec_points)

    mean_list = [np.mean(errors[column]) for column in errors]
    ci_list = [stt.t.interval(ci_int_val,len(errors[column])-1, np.mean(errors[column]), stt.sem(errors[column])) for column in errors]
    errors.loc['Mean'] = np.around(mean_list,dec_points)
    errors.loc['95% CI'] = re_round(ci_list,dec_points)

    mean_list = [np.mean(weights[column]) for column in weights]
    ci_list = [stt.t.interval(ci_int_val,len(weights[column])-1, np.mean(weights[column]), stt.sem(weights[column])) for column in weights]
    weights.loc['Mean'] = np.around(mean_list,dec_points)
    weights.loc['95% CI'] = re_round(ci_list,dec_points)

    # Make plot
    model_plot=plt.figure(figsize=(4,4), dpi=300)
    ax1 = plt.gca()
    ml_plot_train_points=ax1.scatter(y,np.dot(X,w)+intercept,c='k',alpha=1,linewidth=0)
    ax1.plot([-273,2000],[-273,2000],'k--',lw=1)
    ml_plot_test_points=ax1.scatter(y_test,np.dot(X_test,w)+intercept,c='r',alpha=1,linewidth=0)
    lims = [min(y+y_test)-15,max(y+y_test)+15]
    for theaxis in [ax1]:        
        theaxis.set_aspect(1)
        theaxis.set_xlim(lims)
        theaxis.set_ylim(lims)
        theaxis.set_xlabel(r"Experimental $T_m$ ($^{\circ}$C)")
        theaxis.set_ylabel(r"Predicted $T_m$ ($^{\circ}$C)")
        
        for item in ([theaxis.xaxis.label, theaxis.yaxis.label, theaxis.yaxis.get_offset_text(), theaxis.xaxis.get_offset_text()]):
            item.set_fontsize(12)
        for item in (theaxis.get_xticklabels() + theaxis.get_yticklabels()):
            item.set_fontsize(10)
    plt.legend((ml_plot_train_points,ml_plot_test_points),('Training Set','Test Set'))    
    plt.gcf().subplots_adjust(left=0.2,top=0.95,bottom=0.15,right=0.95)
   
    plt.close()
    
    ml_model_dict={
        'Plot': model_plot,
        'Errors': errors,
        'Model Coefficients': weights
    }
    return ml_model_dict

def rf_model_vbt_features_tau_inputs_multipleruns(vbt_dataset_dict,dataset_key,number_of_runs=5):
    
    full_dataset = vbt_dataset_dict[dataset_key][['sigma','SP3 (LIN)','SP3 (BR)','SP2','Ring','V_m (nm3)','T_m (K)','Eccentricity(Ear)','Eccentricity(Eal)']]

    train_avg_err=[]
    test_avg_err=[]
    train_rmse_err=[]
    test_rmse_err=[]
    runs = ["Run " + str(i+1) for i in range(number_of_runs)]
    weights = pd.DataFrame(columns = ['sigma','SP3 (LIN)','SP3 (BR)','SP2','Ring','V_m (nm3)','Eccentricity(Ear)','Eccentricity(Eal)'],index=runs)

    count = 1
    while count < number_of_runs + 1:
        [trainset,testset] = split_data(full_dataset)
        y = (trainset['T_m (K)']-273).tolist()
        y_test = (testset['T_m (K)']-273).tolist()
        trainset = trainset.drop(columns = ['T_m (K)'])
        testset = testset.drop(columns = ['T_m (K)'])
        X = preprocessing.scale(trainset)
        scaler = preprocessing.StandardScaler().fit(trainset)
        X_test = scaler.transform(testset)
        rf = RandomForestRegressor()
        rf.fit(X, y)
        # w = rr.coef_
        # intercept = rr.intercept_
        
        train_avg_err.append(np.mean(np.abs(y-(rf.predict(X)))))
        test_avg_err.append(np.mean(np.abs(y_test-(rf.predict(X_test)))))
        train_rmse_err.append(np.sqrt(((y - (rf.predict(X))) ** 2).mean()))
        test_rmse_err.append(np.sqrt(((y_test-(rf.predict(X_test)))**2).mean()))
        
        weights.loc["Run " + str(count),:] = rf.feature_importances_
        count = count + 1
    
    ci_int_val=0.95
    dec_points=3

    errors = pd.DataFrame(list(zip(train_avg_err,test_avg_err,train_rmse_err,test_rmse_err)),columns=['Train AAE','Test AAE','Train RMSE','Test RMSE'],index=runs).round(decimals=dec_points)

    mean_list = [np.mean(errors[column]) for column in errors]
    ci_list = [stt.t.interval(ci_int_val,len(errors[column])-1, np.mean(errors[column]), stt.sem(errors[column])) for column in errors]
    errors.loc['Mean'] = np.around(mean_list,dec_points)
    errors.loc['95% CI'] = re_round(ci_list,dec_points)

    mean_list = [np.mean(weights[column]) for column in weights]
    ci_list = [stt.t.interval(ci_int_val,len(weights[column])-1, np.mean(weights[column]), stt.sem(weights[column])) for column in weights]
    weights.loc['Mean'] = np.around(mean_list,dec_points)
    weights.loc['95% CI'] = re_round(ci_list,dec_points)

    # Make plot
    model_plot=plt.figure(figsize=(4,4), dpi=300)
    ax1 = plt.gca()
    ml_plot_train_points=ax1.scatter(y,rf.predict(X),c='k',alpha=1,linewidth=0)
    ax1.plot([-273,2000],[-273,2000],'k--',lw=1)
    ml_plot_test_points=ax1.scatter(y_test,rf.predict(X_test),c='r',alpha=1,linewidth=0)
    lims = [min(y+y_test)-15,max(y+y_test)+15]
    for theaxis in [ax1]:        
        theaxis.set_aspect(1)
        theaxis.set_xlim(lims)
        theaxis.set_ylim(lims)
        theaxis.set_xlabel(r"Experimental $T_m$ ($^{\circ}$C)")
        theaxis.set_ylabel(r"Predicted $T_m$ ($^{\circ}$C)")
        
        for item in ([theaxis.xaxis.label, theaxis.yaxis.label, theaxis.yaxis.get_offset_text(), theaxis.xaxis.get_offset_text()]):
            item.set_fontsize(12)
        for item in (theaxis.get_xticklabels() + theaxis.get_yticklabels()):
            item.set_fontsize(10)
    plt.legend((ml_plot_train_points,ml_plot_test_points),('Training Set','Test Set'))    
    plt.gcf().subplots_adjust(left=0.2,top=0.95,bottom=0.15,right=0.95)
   
    plt.close()
    
    ml_model_dict={
        'Plot': model_plot,
        'Errors': errors,
        'Feature Importances': weights
    }
    return ml_model_dict




In [5]:
quinone_data = pd.read_csv("Data Files/Entropy and Volume Data - Quinones.csv")
hydroquinone_data= pd.read_csv("Data Files/Entropy and Volume Data - Hydroquinones.csv")
hydrocarbon_data= pd.read_csv("Data Files/Entropy and Volume Data - Hydrocarbons.csv")
mega_database=pd.concat([hydroquinone_data,quinone_data])[['sigma','tau','V_m (nm3)','T_m (K)','Eccentricity(Ear)','Eccentricity(Eal)']].reset_index(drop=True)
# ML Data
quinone_ML_data=pd.read_csv('Data Files/parsed_p_benzoquinone_216.csv')
hydroquinone_ML_data=pd.read_csv('Data Files/parsed_p_hydroquinone_204.csv')
# Make dictionary for VBT data
dataset_dict={'Quinones':quinone_data,'Hydroquinones':hydroquinone_data,'Hydrocarbons':hydrocarbon_data,'Quinones + Hydroquinones':mega_database}
# Make dictionary for ML data
ml_dataset_dict={'Quinones':quinone_ML_data.drop(columns='Unnamed: 0'),'Hydroquinones':hydroquinone_ML_data.drop(columns='Unnamed: 0')}

In [204]:
ml_dict = rf_model_vbt_features_tau_inputs_multipleruns(dataset_dict,"Hydroquinones",number_of_runs=100)

In [208]:
ml_dict['Plot']
# ml_dict['Plot'].savefig('Plots/RandomForest_HQ_VBTFeatures_SepTau_plot.png',dpi=300,bbox_inches="tight")

In [209]:
ml_dict['Errors']
# ml_dict["Errors"].to_csv('Errors/RandomForest_HQ_VBTFeatures_SepTau_Errors.csv')

In [210]:
ml_dict["Feature Importances"]
# ml_dict["Feature Importances"].to_csv('Parameters/RandomForest_HQ_VBTFeatures_SepTau_Parameters.csv')

## Thermodynamic Model

In [151]:
model_form= '(parameters[0]*predictors["V_m (nm3)"]**(-1)+parameters[1])/(parameters[2]*np.log(predictors["sigma"])+parameters[3]*predictors["tau"]+1+parameters[4]*np.log(predictors["Eccentricity(Ear)"])+parameters[5]*np.log(predictors["Eccentricity(Eal)"]))'

starting_guesses= [0,300,-0.01,0.01,-0.01,-0.01]
dataset_name='Hydrocarbons'
num_runs=100
vbt_bq_dict=vbt_model_automated(dataset_dict,dataset_name,model_form,starting_guesses,num_runs)


In [155]:
vbt_bq_dict['Parameters'].rename(columns={0:'g',1:'h',2:'a',3:'b',4:'c',5:'d'})
# .to_csv("Parameters/Thermodynamic_HC_Parameters.csv")

In [156]:
vbt_bq_dict["Errors"]
# .to_csv("Errors/Thermodynamic_HC_Errors.csv")

In [157]:
vbt_bq_dict["Plot"]
# .savefig('Plots/Thermodynamic_HC_plot.png',dpi=300,bbox_inches="tight")